In [6]:
from lib.config import Config
from lib.data_set import Dataset
from lib.model import NNModel
from lib import utils

In [19]:
import ctypes
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras import layers as KL
from tensorflow.keras import models as KM
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow import keras
import tensorflow as tf

## File name read

In [8]:
# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)
# player = ctypes.windll.kernel32

ind = df2.iloc[1].isna()
files = np.concatenate([np.array(df.columns),np.array('normal/'+df2.columns[ind])])

## Configuration

In [9]:
# Override the base class of Config and Features for CWT-CNN Model
class CWTCNN_Config(Config):
    NAME = 'CWT_CNN'
    NUM_CLASSES = 2
    EPOCHS = 300
    BATCH_SIZE = 32
    CWT_WAVELET = 'mexh'
    CWT_SCALE = 32
    CLASS_WEIGHTS = None
    FN_LP = 300
    DETREND_LAMBDA = 50
    TEST_FILES = files[[6,30,31,32,33,34,35]]
    
class CWT_dataset(Dataset):
    
    def __init__(self,config):
        super(CWT_dataset,self).__init__(config)
        self.cwt_wavelet = config.CWT_WAVELET
        self.cwt_scale = config.CWT_SCALE
        self.config = config
        

In [10]:
# Generate CWT-CNN configuration
config = CWTCNN_Config()
config.display()


Configurations:
BATCH_SIZE                     32
CHANNELS                       ['LEFT_TA', 'LEFT_TS', 'LEFT_BF', 'LEFT_RF', 'RIGHT_TA', 'RIGHT_TS', 'RIGHT_BF', 'RIGHT_RF']
CLASS_WEIGHTS                  None
CWT_SCALE                      32
CWT_WAVELET                    mexh
DETREND_LAMBDA                 50
DROP_WITH_ZSCORE               None
EPOCHS                         300
FN_HP                          None
FN_IR                          False
FN_LP                          300
NAME                           CWT_CNN
NUM_CLASSES                    2
N_ENV                          20
RECT                           False
REMOVE_FREQS                   True
SAME_LABEL                     True
SAVE                           False
SCALE                          True
SHUFFLE                        True
STEP_SIZE                      512
TEST_FILES                     ['G08_FoG_1_trial_1_emg.csv' 'normal/G09_Walking_trial_2_emg.csv'
 'normal/G09_Walking_trial_4_emg.csv' 'normal/G09_

## Data generate

In [11]:
data = CWT_dataset(config)

In [12]:
# Load data from files
data.load_data(files)

X_train,Y_train,_ = data.train_set
X_valid,Y_valid,_ = data.valid_set
X_test, Y_test, _ = data.test_set

skip
skip
3/174: G06_FoG_trial_1_emg.csv
4/174: G06_FoG_trial_2_emg.csv
5/174: G06_FoG_trial_3_emg.csv
6/174: G07_Freezing_Trial1_trial_1_emg.csv
7/174: G08_FoG_1_trial_1_emg.csv
8/174: G08_FoG_2_trial_1_emg.csv
9/174: G11_FoG_trial_1_emg.csv
10/174: G11_FoG_trial_2_emg.csv
11/174: P379_M050_2_OFF_A_FoG_trial_1_emg.csv
12/174: P379_M050_2_OFF_A_FoG_trial_2_emg.csv
13/174: P379_M050_2_OFF_A_FoG_trial_3_emg.csv
14/174: P379_M050_2_OFF_B_FoG_trial_1_emg.csv
15/174: P379_M050_2_OFF_B_FoG_trial_2_emg.csv
16/174: P379_M050_2_OFF_B_FoG_trial_3_emg.csv
17/174: P551_M050_2_A_FoG_trial_1_emg.csv
18/174: P551_M050_2_B_FoG_trial_1_emg.csv
19/174: P551_M050_2_B_FoG_trial_2_emg.csv
20/174: P812_M050_2_B_FoG_trial_1_emg.csv
21/174: P812_M050_2_B_FoG_trial_2_emg.csv
22/174: normal/G02_Walking_trial_1_emg.csv
23/174: normal/G03_Walking_trial_1_emg.csv
24/174: normal/G03_Walking_trial_2_emg.csv
25/174: normal/G05_Walking_struct_fixed_trial_1_emg.csv
26/174: normal/G05_Walking_struct_fixed_trial_2_emg.cs

171/174: normal/P940_MSham_A_Walking_trial_6_emg.csv
172/174: normal/P940_MSham_B_Walking_trial_2_emg.csv
173/174: normal/P940_MSham_B_Walking_trial_4_emg.csv
174/174: normal/P940_MSham_B_Walking_trial_6_emg.csv


## Model

In [13]:
# Override base class of SimpleMode for CWT_CNN
class CWT_CNN_Model(NNModel):
    
    def build(self,config):
        
        kernel = 3
        acti = 'elu'
        stride = 1
        pool_size = (2,2)
        drop_rate = 0
        
        wavelet = config.CWT_WAVELET
        scale = config.CWT_SCALE
        fc = pywt.central_frequency(wavelet)
        cparam = 2 * fc * scale
        scales = cparam / np.arange(int(scale+1), 1, -1)
        
        input_ = KL.Input(shape=[config.WINDOW_SIZE,len(config.CHANNELS)])
        cwt = KL.Lambda(lambda x:utils.cwt_tf(x,scales,wavelet,config.BATCH_SIZE,1e-3),name='cwt')(input_)

        cnn1 = KL.Conv2D(128,kernel,strides=stride,
                              kernel_initializer=TruncatedNormal(),
                              #use_bias=False,#activation='elu',
                              padding='same',
                              name='conv1')(cwt)
        cnn1 = KL.Activation(acti,name='act_conv1')(cnn1)
        cnn1 = KL.MaxPooling2D(pool_size,name='max_pool_conv1')(cnn1)
        cnn1 = KL.Dropout(drop_rate,name='dropout_conv1')(cnn1)

        cnn2 = KL.Conv2D(64,kernel,strides=stride,
                              kernel_initializer=TruncatedNormal(),
                              #use_bias=False,#activation='elu',
                              padding='same',
                              name='conv2')(cnn1)
        cnn2 = KL.Activation(acti,name='act_conv2')(cnn2)
        cnn2 = KL.MaxPooling2D(pool_size,name='max_pool_conv2')(cnn2)
        cnn2 = KL.Dropout(drop_rate,name='dropout_conv2')(cnn2)

        cnn3 = KL.Conv2D(32,kernel,strides=stride,
                              kernel_initializer=TruncatedNormal(),
                              #use_bias=False,#activation='elu',
                              padding='same',
                              name='conv3')(cnn2)
        cnn3 = KL.Activation(acti,name='act_conv3')(cnn3)
        cnn3 = KL.MaxPooling2D(pool_size,name='max_pool_conv3')(cnn3)
        cnn3 = KL.Dropout(drop_rate,name='dropout_conv3')(cnn3)

        cnn4 = KL.Conv2D(16,kernel,strides=stride,
                              kernel_initializer=TruncatedNormal(),
                              #use_bias=False,#activation='elu',
                              padding='same',
                              name='conv4')(cnn3)
        cnn4 = KL.Activation(acti,name='act_conv4')(cnn4)
        cnn4 = KL.MaxPooling2D(pool_size,name='max_pool_conv4')(cnn4)
        cnn4 = KL.Dropout(drop_rate,name='dropout_conv4')(cnn4)

        cnn5 = KL.Conv2D(8,1,strides=stride,
                              kernel_initializer=TruncatedNormal(),
                              #use_bias=False,#activation='elu',
                              padding='same',
                              name='conv5')(cnn4)
        cnn5 = KL.Activation(acti,name='act_conv5')(cnn5)
        cnn5 = KL.Dropout(drop_rate,name='dropout_conv5')(cnn5)

        flatten = KL.GlobalAveragePooling2D(name='gloabel_average')(cnn5)

        dropout = KL.Dropout(drop_rate)(flatten)

        output = KL.Dense(config.NUM_CLASSES,activation = 'softmax',name='output_cnn')(dropout)
        model = KM.Model(inputs=[input_],outputs=[output])
        
        # model.summary()
        if config.COST_SENSITIVE:
            self.cost_matrix = config.COST_MATRIX
            model.compile(loss=self.sparse_cost_sensitive_loss, optimizer="adam", metrics=['accuracy'])
            print('Using cost sensitive with cost matrix:\n',np.array(self.cost_matrix))
        else:
            model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
            if config.CLASS_WEIGHTS != None:
                print('Using categorical crossentropy with class weights:\n',config.CLASS_WEIGHTS)
            else:
                print('Using categorical crossentropy without class weights.')
        
        return model

    def sparse_cost_sensitive_loss (self,y_true,y_pred):
        cost_matrix = self.cost_matrix
        batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
        eps = 1e-6
        probability = tf.clip_by_value(y_pred, eps, 1-eps)
        cost_values = tf.math.log(1-probability)*batch_cost_matrix
        loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
        return loss

    def model_metrics(self,data,label):
        pred = self.keras_model.predict(data)
        acc = metrics.accuracy_score(np.argmax(label,axis=1),np.argmax(pred,axis=1))
        cm = metrics.confusion_matrix(np.argmax(label,axis=1),np.argmax(pred,axis=1))
        f1 = metrics.f1_score(np.argmax(label,axis=1),np.argmax(pred,axis=1),average='macro')
        return acc,cm,f1

## Data split

In [14]:
# data split and processing for model
class_id = [1,2,6]
binary = False
x_train,y_train,x_valid,y_valid,x_test,y_test,oh = utils.data_split_oh((X_train,X_valid,X_test),
                                                                    (Y_train,Y_valid,Y_test),
                                                                    class_id,
                                                                    binary,
                                                                    random_state = 555)

## Model training

In [16]:
config.COST_MATRIX = tf.constant([[0,1.],
              [10,0]])

if binary:
    config.COST_SENSITIVE = True
    config.NUM_CLASSES = 2
else:
    config.COST_SENSITIVE = False
    config.NUM_CLASSES = len(class_id)

# Generate CWT_CNN Model
cwt_model = CWT_CNN_Model('CWT_CNN',config,'./model/CWT_CNN/')

Using categorical crossentropy without class weights.


In [17]:
early_stopping = keras.callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             #baseline = 0.9,
                                             restore_best_weights=True)
cwt_model.train((x_train,y_train),(x_valid,y_valid),config.EPOCHS,config.BATCH_SIZE,[early_stopping])


Starting at epoch 0.

Checkpoint Path: ./model/CWT_CNN/cwt_cnn20211105T0017\CWT_CNN_cwt_cnn_{epoch:04d}.h5
Epoch 1/300
14/14 [==============================] - 65s 4s/step - loss: 1.0397 - accuracy: 0.4659 - val_loss: 0.8809 - val_accuracy: 0.6507
Epoch 2/300
14/14 [==============================] - 64s 5s/step - loss: 0.9485 - accuracy: 0.5632 - val_loss: 0.7220 - val_accuracy: 0.6575
Epoch 3/300
14/14 [==============================] - 62s 4s/step - loss: 0.8168 - accuracy: 0.6267 - val_loss: 0.6609 - val_accuracy: 0.7671
Epoch 4/300
14/14 [==============================] - 60s 4s/step - loss: 0.7679 - accuracy: 0.6529 - val_loss: 0.6002 - val_accuracy: 0.7945
Epoch 5/300
14/14 [==============================] - 64s 5s/step - loss: 0.6904 - accuracy: 0.7131 - val_loss: 0.5637 - val_accuracy: 0.7740
Epoch 6/300
14/14 [==============================] - 60s 4s/step - loss: 0.6765 - accuracy: 0.7005 - val_loss: 0.5929 - val_accuracy: 0.7808
Epoch 7/300
14/14 [===========================

Epoch 58/300
14/14 [==============================] - 52s 4s/step - loss: 0.1215 - accuracy: 0.9585 - val_loss: 0.5199 - val_accuracy: 0.8562
Epoch 59/300
14/14 [==============================] - 52s 4s/step - loss: 0.1287 - accuracy: 0.9473 - val_loss: 0.4802 - val_accuracy: 0.8425


## Model evaluation

In [20]:
acc_train,cm_train,f1_train = cwt_model.model_metrics(x_train,y_train)
acc_valid,cm_valid,f1_valid = cwt_model.model_metrics(x_valid,y_valid)
acc_test,cm_test,f1_test = cwt_model.model_metrics(x_test,y_test)
print('acc_train: %f\nf1_train: %f\nconfusion_matrix:\n'%(acc_train,f1_train),cm_train)
print('acc_valid: %f\nf1_valid: %f\nconfusion_matrix:\n'%(acc_valid,f1_valid),cm_valid)
print('acc_test: %f\nf1_test: %f\nconfusion_matrix:\n'%(acc_test,f1_test),cm_test)

acc_train: 0.874429
f1_train: 0.823112
confusion_matrix:
 [[ 45  22  18]
 [  1 239   6]
 [  2   6  99]]
acc_valid: 0.849315
f1_valid: 0.707099
confusion_matrix:
 [[ 5  7  7]
 [ 3 88  4]
 [ 1  0 31]]
acc_test: 0.718121
f1_test: 0.678276
confusion_matrix:
 [[14 20  1]
 [ 8 45  0]
 [12  1 48]]
